In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import re

In [ ]:
##Create list of all structured files (from EPIC) and list with names of files
print('start0')
path_to_dir=r'Z:\final EPIC data 20220909\*.xlsx'
list_files=glob.glob(path_to_dir)
data_all=[]
file_name=[]
print('start1')
print(len(list_files))
for j in range(0,len(list_files)):
    print('j:', j)
    data_all.append(pd.read_excel(list_files[j]))
    file_name.append(list_files[j][32:-5])

print('start2')
##Read NKR registry data
NKR_data=pd.read_csv(r"Z:\IKNL data\IKNL data update\K21186.csv", delimiter=';')

print('start3')
##Read tables with mapping between Patient ID (from the structured files) to the pseudoID of the clinical notes
AMC_conv=pd.read_excel(r"Z:\final EPIC data 20220909\Text data 6-10-2022\002a AMC CTcue import-results-2022-09-12.xlsx")
VUmc_conv=pd.read_excel(r"Z:\final EPIC data 20220909\Text data 6-10-2022\002b VUmc CTcue import-results-2022-09-09.xlsx")

print('start4')
##Read clinical notes from location AMC and location VUmc
notes_AMC=pd.read_csv(r"Z:\final EPIC data 20220909\Text data 6-10-2022\amc_data_aanvraag_20221004_aangepast.csv", encoding="utf-16")
notes_VUmc=pd.read_csv(r"Z:\final EPIC data 20220909\Text data 6-10-2022\vumc_data_aanvraag_20221004_aangepast.csv", encoding="utf-16")

In [ ]:
#From clinical notes pseudo_id to Epic ID (data_all) structured data to NKR ID
Clinical_notes_ID=notes_AMC['pseudo_id'].iloc[0] #example, Patient pseudo ID relative at clinical note at row 0, from location AMC
EPIC_ID=AMC_conv[AMC_conv["Patient_pseudo_id"]==Clinical_notes_ID].Patient_hospital_id.values[0] #ID to be used for all structured data in data_all
NKR_ID=data_all[0][data_all[0].MDN_NKR==EPIC_ID].KEY_NKR.values[0]

#From NKR ID to EPIC ID to clinical notes pseudo_id 

Clinical_notes_ID=notes_AMC['pseudo_id'].iloc[0] #example, Patient pseudo ID relative at clinical note at row 0, from location AMC

####EPIC (structured data, Electronic Health Records): this ID is used only to map to notes and to check content of data in EPIC app (NOT RELEVANT FOR YOU!)
EPIC_ID=AMC_conv[AMC_conv["Patient_pseudo_id"]==Clinical_notes_ID].Patient_hospital_id.values[0] #ID to be used for all structured data in data_all

####NKR:
NKR_ID=data_all[0][data_all[0].MDN_NKR==EPIC_ID].KEY_NKR.values[0] #ID to be used for both NKR and for EPIC data (data_all)
data_patient_NKR=NKR_data[NKR_data.key_nkr==NKR_ID]

#Example: extract lab tests for patient with ID = NKR_ID
data_patient_EPIC_lab=data_all[3][data_all[3].KEY_NKR==NKR_ID] 

In [ ]:
#notes_VUmc['location'] = 'VUMC'
#notes_AMC['location'] = 'AMC'
df_amc_conv = AMC_conv
df_vumc_conv = VUmc_conv
notes_VUmc['location'] = 'VUmc'
notes_AMC['location'] = 'AMC'
df = pd.concat([notes_VUmc, notes_AMC])

In [ ]:
import copy
df_backup = copy.deepcopy(df)
conv = pd.concat([AMC_conv, VUmc_conv])

In [ ]:
df['note_nr'] = range(len(df))

In [ ]:
# Merge the DataFrames on Patient_hospital_id
patient_id_matching = pd.merge(df_amc_conv, df_vumc_conv, on='Patient_hospital_id')
pseudo_id_list = [item for item in df_amc_conv['Patient_pseudo_id']] + [item for item in df_vumc_conv['Patient_pseudo_id']]

print(patient_id_matching.head())
print(pseudo_id_list[:10])

import pandas as pd

def assign_global_pseudo_id(df, patient_id_matching):
  """
  Assigns a global pseudo ID to each row in 'df' based on matching in 'patient_id_matching'.

  Args:
      df (pandas.DataFrame): The DataFrame to modify.
      patient_id_matching (pandas.DataFrame): The DataFrame containing patient ID mappings.

  Returns:
      pandas.DataFrame: The modified DataFrame with the 'global_pseudo_id' column.
  """
  df['global_pseudo_id'] = df['pseudo_id'].apply(
      lambda x: patient_id_matching[patient_id_matching['Patient_pseudo_id_x'] == x]['Patient_hospital_id'].tolist() +
                 patient_id_matching[patient_id_matching['Patient_pseudo_id_y'] == x]['Patient_hospital_id'].tolist()
  )

  # Handle cases where no match is found (empty lists)
  df['global_pseudo_id'] = df['global_pseudo_id'].apply(lambda x: x[0] if x else None)

  return df

# Apply the function to your DataFrame
df = assign_global_pseudo_id(df.copy(), patient_id_matching.copy())
print(df.head())

In [ ]:
print(df['global_pseudo_id'].isna().sum())
print(len(df['global_pseudo_id']) - df['global_pseudo_id'].isna().sum())

In [ ]:
from random import choices
import string

def anonymize_hospital_ids(df):
  """
  An anonymizes hospital IDs in 'df' and generates unique pseudonyms.

  Args:
      df (pandas.DataFrame): The DataFrame containing the 'global_pseudo_id' column.

  Returns:
      tuple: A tuple containing two lists:
          - unique_hospital_ids: List of unique hospital IDs from 'global_pseudo_id'.
          - pseudonymised_ids: List of unique randomly generated 10-character pseudonyms.
  """
  # Get unique hospital IDs
  unique_hospital_ids = df['global_pseudo_id'].dropna().unique().tolist()

  # Define function to generate random pseudonyms
  def generate_unique_pseudonym(used_codes):
    while True:
      pseudonym = ''.join(choices(string.ascii_uppercase + string.digits, k=10))
      if pseudonym not in used_codes:
        return pseudonym

  # Generate unique pseudonyms
  used_codes = set()  # Keep track of used pseudonyms
  pseudonymised_ids = [generate_unique_pseudonym(used_codes) for _ in range(len(unique_hospital_ids))]

  # Create a dictionary for mapping (optional)
  id_mapping = dict(zip(unique_hospital_ids, pseudonymised_ids))

  return unique_hospital_ids, pseudonymised_ids, id_mapping  # Optional: return mapping dictionary

# Apply the function
unique_hospital_ids, pseudonymised_ids, id_mapping = anonymize_hospital_ids(df.copy())

# Print results (optional)
#print(f"Unique Hospital IDs: {unique_hospital_ids}")
#print(f"Pseudonymised IDs: {pseudonymised_ids}")

# You can use the id_mapping for further processing (optional)

In [ ]:
def replace_with_pseudonyms(df, unique_hospital_ids, pseudonymised_ids):
  """
  Replaces hospital IDs in 'df' with corresponding pseudonyms.

  Args:
      df (pandas.DataFrame): The DataFrame containing the 'global_pseudo_id' column.
      unique_hospital_ids (list): List of unique hospital IDs.
      pseudonymised_ids (list): List of pseudonyms corresponding to unique hospital IDs.

  Returns:
      pandas.DataFrame: The modified DataFrame with replaced hospital IDs.
  """

  # Create a dictionary for mapping
  id_mapping = dict(zip(unique_hospital_ids, pseudonymised_ids))

  # Replace hospital IDs with pseudonyms using vectorized approach
  def replace_id(x):
    return id_mapping.get(x)  # Use get to handle potential missing values

  df['global_pseudo_id'] = df['global_pseudo_id'].apply(replace_id)

  return df

# Apply the function
df = replace_with_pseudonyms(df.copy(), unique_hospital_ids, pseudonymised_ids)


In [ ]:
# # Function to replace NaN with sequential numbers
# def replace_nan_with_sequential(df, col_name):
#   nan_count = 0
#   for i in range(len(df)):
#     if pd.isna(df.loc[i, col_name]):
#       df.loc[i, col_name] = f"NaN{nan_count}"
#       nan_count += 1
#   return df

# # Replace NaN values in the 'global_pseudo_id' column
# df['global_pseudo_id'] = df['global_pseudo_id'].fillna(np.nan)
# df = replace_nan_with_sequential(df.copy(), "global_pseudo_id")

# # Print the dataframe
# print(df.head(100))

def replace_with_pseudo_id(df, col1, col2):
    """
    Replaces NaN values in 'col1' with corresponding values from 'col2'.

    Args:
        df (pd.DataFrame): The dataframe to modify.
        col1 (str): The column name containing NaN values.
        col2 (str): The column name containing values to use for replacement.

    Returns:
        pd.DataFrame: The modified dataframe.
    """

    return df.assign(**{col1: df[col1].fillna(df[col2])})

# Replace NaN values
df['global_pseudo_id'] = df['global_pseudo_id'].fillna(np.nan)
df = replace_with_pseudo_id(df.copy(), "global_pseudo_id", "pseudo_id")

# Print the dataframe
print(df)

In [ ]:
# amount of notes
print(len(df))
# amount of patients
print(df['global_pseudo_id'].nunique())
print(df['pseudo_id'].nunique())

In [ ]:
def replace_ids(df, pid_list):
  """Replaces pids in a dataframe with their corresponding global_pseudo_ids

  Args:
      df (pandas.DataFrame): The dataframe containing the pseudo_id and global_pseudo_id columns
      pid_list (list): A list of pseudo_ids to be replaced

  Returns:
      list: A list containing the replaced global_pseudo_ids with the same length as pid_list
  """
  # Create a dictionary to map pids to global_pseudo_ids
  pid_map = dict(zip(df['pseudo_id'], df['global_pseudo_id']))

  # Handle potential length mismatch
  if len(pid_list) > len(df):
      pid_list = pid_list[:len(df)]  # Shorten pid_list if longer

  # Replace pids in the list with their corresponding global_pseudo_ids, assigning NaN for missing values
  replaced_list = [pid_map.get(pid) for pid in pid_list]
  replaced_list = [val if val is not None else np.NAN for val in replaced_list]

  # Truncate the replaced_list to match the length of pid_list
  replaced_list = replaced_list[:len(pid_list)]

  return replaced_list

In [ ]:
print(df.columns)

In [ ]:
import random
#AMC = df.loc[df['location'] == 'AMC']
#VUMC = df.loc[df['location'] == 'VUmc']
#AMC_100 = random.sample(AMC['pseudo_id'].unique().tolist(), 100)
#VUMC_100 = random.sample(VUMC['pseudo_id'].unique().tolist(), 100)
sample_50_ids = random.sample(df['pseudo_id'].unique().tolist(), 50)
sample_50_ids = df[df['pseudo_id'].isin(sample_50_ids)]

In [ ]:
sample_50_ids.reset_index(drop = True, inplace = True)
print(sample_50_ids.head(2))
print(sample_50_ids.columns)
print(sample_50_ids['pseudo_id'].nunique())
print(len(sample_50_ids))

In [ ]:
Clinical_notes_ID=sample_50_ids['pseudo_id'].iloc[1] #example, Patient pseudo ID relative at clinical note at row 0, from location AMC
EPIC_ID=conv[conv["Patient_pseudo_id"]==Clinical_notes_ID].Patient_hospital_id.values[0] #ID to be used for all structured data in data_all
NKR_ID=data_all[0][data_all[0].MDN_NKR==EPIC_ID].KEY_NKR.values[0]

print(Clinical_notes_ID)
print(EPIC_ID)
print(NKR_ID)

In [ ]:
for i in range(len(sample_50_ids[0:10])):
    print(sample_50_ids['pseudo_id'].iloc[i])

In [ ]:
NKR_list = []

for i in range(len(sample_50_ids)):
    Clinical_notes_ID=sample_50_ids['pseudo_id'].iloc[i] #example, Patient pseudo ID relative at clinical note at row 0, from location AMC
    EPIC_ID=conv[conv["Patient_pseudo_id"]==Clinical_notes_ID].Patient_hospital_id.values[0] #ID to be used for all structured data in data_all
    NKR_ID=data_all[0][data_all[0].MDN_NKR==EPIC_ID].KEY_NKR.values[0]
    NKR_list.append(NKR_ID)
    
sample_50_ids['KEY_NKR'] = NKR_list

In [ ]:
opname_df = data_all[29][['KEY_NKR', 'OPNAMEMOMENT', 'ONTSLAGMOMENT']]

In [ ]:
#sample_50_ids = pd.merge(sample_50_ids, opname_df, on='KEY_NKR', how='inner')

In [ ]:
opname_df = opname_df[opname_df['KEY_NKR'].isin(sample_50_ids['KEY_NKR'])]

In [ ]:

opname_df['OPNAMEMOMENT'] = pd.to_datetime(opname_df['OPNAMEMOMENT'])
opname_df['OPNAMEMOMENT'] = opname_df['OPNAMEMOMENT'].dt.date

opname_df['ONTSLAGMOMENT'] = pd.to_datetime(opname_df['ONTSLAGMOMENT'])
opname_df['ONTSLAGMOMENT'] = opname_df['ONTSLAGMOMENT'].dt.date

In [ ]:
# Group by 'KEY_NKR' and aggregate the dates into lists
result_df = opname_df.groupby('KEY_NKR').agg({
    'OPNAMEMOMENT': lambda x: list(x),
    'ONTSLAGMOMENT': lambda x: list(x)
}).reset_index()

# Rename columns
result_df.columns = ['KEY_NKR', 'OPNAMEMOMENTEN', 'ONTSLAGMOMENTEN']

print(result_df)

opname_df = result_df

In [ ]:
sample_50_ids = pd.merge(sample_50_ids, opname_df, on='KEY_NKR', how='inner')

In [ ]:
sample_50_ids['report_date'] = pd.to_datetime(sample_50_ids ['report_date'])
sample_50_ids['report_date'] = sample_50_ids ['report_date'].dt.date


In [ ]:
# Function to check if report_date falls within hospital stays, with exception for same day admission and discharge
def is_report_date_within_hospital_stay(row):
    print(row)
    report_date = row['report_date']
    opname_momenten = pd.to_datetime(row['OPNAMEMOMENTEN'])
    ontslag_momenten = pd.to_datetime(row['ONTSLAGMOMENTEN'])
    
    for opname, ontslag in zip(opname_momenten, ontslag_momenten):
        if opname <= report_date <= ontslag:
            if opname != ontslag:
                return True
    return False

# Filter out rows where the report_date falls within any hospital stay period
filtered_sample_50_ids = sample_50_ids[~sample_50_ids.apply(is_report_date_within_hospital_stay, axis=1)]

# Calculate the number of same-day admission and discharge scenarios that match the report_date
same_day_count = 0
for index, row in sample_50_ids.iterrows():
    report_date = row['report_date']
    opname_momenten = pd.to_datetime(row['OPNAMEMOMENTEN'])
    ontslag_momenten = pd.to_datetime(row['ONTSLAGMOMENTEN'])
    for opname, ontslag in zip(opname_momenten, ontslag_momenten):
        if opname == ontslag and opname == report_date:
            same_day_count += 1

#print(filtered_valtest)
print(f"Number of same-day admission and discharge scenarios matching report_date: {same_day_count}")

In [ ]:
sample_50_ids = filtered_sample_50_ids

In [ ]:
sample_50_ids = sample_50_ids[sample_50_ids['report_type'].isin(['Voortgangsverslag', 'MDO-verslag', 'Anesthesie pre-op'])]

In [ ]:
sample_50_ids['report_type'].unique()

In [ ]:
AMC = sample_50_ids.loc[sample_50_ids['location'] == 'AMC']
VUMC = sample_50_ids.loc[sample_50_ids['location'] == 'VUmc']

In [ ]:
print(AMC['report_type'].unique())
print(VUMC['report_type'].unique())

In [ ]:
df = sample_50_ids

# Remove double ID

In [ ]:
#remove from sample any double ID's
sample_selec1 = pd.read_csv('ann1_first_round.csv')
sample_selec2 = pd.read_csv('ann2_first_round.csv')
sample_selec = pd.concat([sample_selec1, sample_selec2], ignore_index=True)

df = df[~df['pseudo_id'].isin(sample_selec['pseudo_id'])]


# Remove files =< 7 sentences

In [ ]:
!pip install spacy
import pandas as pd
import spacy
from spacy.cli import download

# Load the spaCy model for Dutch
try:
    nlp = spacy.load("nl_core_news_sm")
except OSError:
    download("nl_core_news_sm")
    nlp = spacy.load("nl_core_news_sm")
    
# Function to count sentences in a given text using spaCy
def count_sentences(text):
    doc = nlp(text)
    return len(list(doc.sents))

# Apply the function to the 'pseudonymized_text' column and filter rows
df['sentence_count'] = df['pseudonomised_text'].apply(count_sentences)
df = df[df['sentence_count'] > 7].drop(columns='sentence_count')



# Remove 80% fuzzymatched files

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Function to find clusters of similar strings
def find_similar_clusters(df, threshold=80):
    clusters = []
    texts = df['pseudonomised_text'].tolist()
    indices = df.index.tolist()

    while texts:
        text = texts.pop(0)
        index = indices.pop(0)
        cluster = [(index, text)]
        to_remove = []
        for other_index, other_text in zip(indices, texts):
            if fuzz.ratio(text, other_text) >= threshold:
                cluster.append((other_index, other_text))
                to_remove.append((other_index, other_text))
        for item in to_remove:
            texts.remove(item[1])
            indices.remove(item[0])
        clusters.append(cluster)
    
    return clusters

# Function to remove shorter strings within clusters
def remove_shorter_versions(clusters):
    longest_indices = []
    for cluster in clusters:
        longest_index, longest_text = max(cluster, key=lambda x: len(x[1]))
        longest_indices.append(longest_index)
    return longest_indices

# Find clusters of similar strings
clusters = find_similar_clusters(df)

# Remove shorter strings within each cluster
longest_indices = remove_shorter_versions(clusters)

# Create a new DataFrame with only the rows corresponding to the longest texts
df = df.loc[longest_indices]



# Remove files based on (sub)tokens

In [ ]:
df = df[df['pseudonomised_text'].apply(lambda x: all(s in str(x).lower() for s in ['gebeld']))]
df = df[df['pseudonomised_text'].apply(lambda x: all(s in str(x).lower() for s in ['gesproken']))]
df = df[df['pseudonomised_text'].apply(lambda x: all(s in str(x).lower() for s in ['huisarts']))]

# Sample val set

In [ ]:
sampled = []

# Find the lowest number of unique pseudo-ids
x = min(AMC['pseudo_id'].nunique(), VUMC['pseudo_id'].nunique())

# Desired number of rows and tolerance
s = 50
tolerance = 0.05

while True:
  # Sample a number of unique pseudo-ids between 0 and x (inclusive)
  n = random.randint(0, x)

  # Sample n unique pseudo-ids from each dataframe
  sampled_ids_amc = random.sample(AMC['pseudo_id'].unique().tolist(), n)
  sampled_ids_vumc = random.sample(VUMC['pseudo_id'].unique().tolist(), n)
  sampled_global_ids_amc = replace_ids(sample_50_ids.copy(), sampled_ids_amc)
  sampled_global_ids_vumc = replace_ids(sample_50_ids.copy(), sampled_ids_vumc)

   # Count occurrences in each dataframe
  count_amc = sample_50_ids[sample_50_ids['global_pseudo_id'].isin(sampled_global_ids_amc)].shape[0]
  count_vumc = sample_50_ids[sample_50_ids['global_pseudo_id'].isin(sampled_global_ids_vumc)].shape[0]
  sampled_global_total = sampled_global_ids_amc + sampled_global_ids_vumc
  count_total = sample_50_ids[sample_50_ids['global_pseudo_id'].isin(sampled_global_total)].shape[0]

  # Total number of selected pseudo-ids
  total_selected_amount = count_total
  #print('total_selected_amount:', total_selected_amount)
    
  # Check if total_selected_amount is within the desired range
  #if (total_selected_amount >= (1 - tolerance) * s) and (total_selected_amount <= (1 + tolerance) * s):
  if (total_selected_amount >= s) and (total_selected_amount <= (1 + tolerance) * s):
    print('total_selected_amount:', total_selected_amount)
    break

# Collect counts for each pseudo-id in each dataframe
count_dict_amc = AMC[AMC['pseudo_id'].isin(sampled_ids_amc)]['pseudo_id'].value_counts().to_dict()
count_dict_vumc = VUMC[VUMC['pseudo_id'].isin(sampled_ids_vumc)]['pseudo_id'].value_counts().to_dict()

# Print information after reaching desired number
print("VAL set:")
print(f"n: {n}")
print(f"count_amc: {count_amc}")
print(f"count_vumc: {count_vumc}")
print(f"total_selected_amount: {total_selected_amount}")
print(f"count_dict_amc: {count_dict_amc}")
print(f"count_dict_vumc: {count_dict_vumc}")
val_ids = list(count_dict_amc.keys()) + list(count_dict_vumc.keys())

print("Reached desired number of selected pseudo-ids.")

sampled = sampled + val_ids

# Sample test set

In [ ]:
# Find the lowest number of unique pseudo-ids
x = min(AMC['pseudo_id'].nunique(), VUMC['pseudo_id'].nunique())

# Desired number of rows and tolerance
s = 50
tolerance = 0.05

while True:
  # Sample a number of unique pseudo-ids between 0 and x (inclusive)
  n = random.randint(0, x)

  # Sample n unique pseudo-ids from each dataframe
  sampled_ids_amc = random.sample(AMC['pseudo_id'].unique().tolist(), n)
  sampled_ids_vumc = random.sample(VUMC['pseudo_id'].unique().tolist(), n)
  sampled_global_ids_amc = replace_ids(sample_50_ids.copy(), sampled_ids_amc)
  sampled_global_ids_vumc = replace_ids(sample_50_ids.copy(), sampled_ids_vumc)

  # Count occurrences in each dataframe
  count_amc = sample_50_ids[sample_50_ids['global_pseudo_id'].isin(sampled_global_ids_amc)].shape[0]
  count_vumc = sample_50_ids[sample_50_ids['global_pseudo_id'].isin(sampled_global_ids_vumc)].shape[0]
  sampled_global_total = sampled_global_ids_amc + sampled_global_ids_vumc
  count_total = sample_50_ids[sample_50_ids['global_pseudo_id'].isin(sampled_global_total)].shape[0]

  # Total number of selected pseudo-ids
  total_selected_amount = count_total
  #print('total_selected_amount:', total_selected_amount)
    
  # Check if total_selected_amount is within the desired range
  #if (total_selected_amount >= (1 - tolerance) * s) and (total_selected_amount <= (1 + tolerance) * s):
  if (total_selected_amount >= s) and (total_selected_amount <= (1 + tolerance) * s):
    print('total_selected_amount:', total_selected_amount)
    break

# Collect counts for each pseudo-id in each dataframe
count_dict_amc = AMC[AMC['pseudo_id'].isin(sampled_ids_amc)]['pseudo_id'].value_counts().to_dict()
count_dict_vumc = VUMC[VUMC['pseudo_id'].isin(sampled_ids_vumc)]['pseudo_id'].value_counts().to_dict()

# Print information after reaching desired number
print("TEST set:")
print(f"n: {n}")
print(f"count_amc: {count_amc}")
print(f"count_vumc: {count_vumc}")
print(f"total_selected_amount: {total_selected_amount}")
print(f"count_dict_amc: {count_dict_amc}")
print(f"count_dict_vumc: {count_dict_vumc}")

print("Reached desired number of selected pseudo-ids.")

test_ids = list(count_dict_amc.keys()) + list(count_dict_vumc.keys())

print("Reached desired number of selected pseudo-ids.")

sampled = sampled + test_ids

# Testing for unique global id

In [ ]:
print('Global IDs val set:')
pid_list_val = val_ids
#replaced_list_val = replace_ids(df.copy(), pid_list_val)
replaced_list_val = val_ids

# Check if all values in the replaced list are unique
is_unique = len(set(replaced_list_val)) == len(replaced_list_val)
print(replaced_list_val)
print("Are all values in the list unique?", is_unique)


print('Global IDs test set:')
pid_list_test = test_ids
#replaced_list_val = replace_ids(df.copy(), pid_list_val)
replaced_list_test = test_ids


# Check if all values in the replaced list are unique
is_unique = len(set(replaced_list_test)) == len(replaced_list_test)
print(replaced_list_test)
print("Are all values in the list unique?", is_unique)



print('Val + test all unique?')
# Check if all values in the replaced list are unique
val_test_list = sampled
is_unique = len(set(val_test_list)) == len(val_test_list)
print("Are all values in the list unique?", is_unique)

In [ ]:
print(len(replaced_list_val))

valcount = 0

for test_id in replaced_list_val:
    count = sample_50_ids['global_pseudo_id'].value_counts().get(test_id, 0)
    valcount += count
    print(test_id)
    print(count)

print(valcount)

In [ ]:
print(len(replaced_list_test))

testcount = 0

for test_id in replaced_list_test:
    count = sample_50_ids['global_pseudo_id'].value_counts().get(test_id, 0)
    testcount += count
    print(test_id)
    print(count)
    
print(testcount)

In [ ]:
val_df = sample_50_ids[sample_50_ids['global_pseudo_id'].isin(replaced_list_val)]
test_df = sample_50_ids[sample_50_ids['global_pseudo_id'].isin(replaced_list_test)]

In [ ]:
print(val_df['report_type'].unique())
print(test_df['report_type'].unique())

In [ ]:
val_df['set'] = 'val'
test_df['set'] = 'test'

In [ ]:
valtest_df['report_type'].unique()

In [ ]:
valtest_df = pd.concat([val_df, test_df], axis=0)

In [ ]:
print(valtest_df.head())

In [ ]:
#valtest_df.to_csv('second_round_104_files.csv')

In [ ]:
print(testcount)